# A tutorial for the `xds` data structure for neural recordings

## Background

Over the past decades, the Miller Limb Lab has been collecting physiological signals characterizing neural motor control in animal subjects. The codes and scripts for the processing and analyses of these data have been put together as the [ClassyDataAnalysis](https://github.com/limblab/ClassyDataAnalysis) under `MATLAB` environment. In order to maintain the consistency of data structure and processing pipeline in a different environment, @xuanma proposed to develop `cross-platform data structure` (xds) as an extension of `ClassyDataAnalysis` that can be used in `Python`.

The Miller Limb Lab has been using the equipment produced by [Blackrock Inc.](https://blackrockneurotech.com/) for the acquisition of neural signals. The scripts in `ClassyDataAnalysis` read the data recorded by such equipment and convert them into a format that can be loaded in `MATLAB`. `xds` keeps using these scripts for raw data converting due to a variety of reasons, and related codes could be found in [xds_matlab](https://github.com/limblab/xds/tree/master/xds_matlab).

As general users of the data collected by the Miller Limb Lab, it is not necessary to understand every piece of the codes. Here we recommend you to focus on `xds_python` and create your own analysis codes on top of it. This tutorial will tell you how to load, process and use the data with `xds`.

## Where can I get the codes?

Please use `git clone https://github.com/limblab/xds.git` to get the codes from this [repo](https://github.com/limblab/xds/tree/master).

## What should I do to import xds as a package?

Simply use
```
import sys
sys.path.append('your own path/xds/xds_python/') 

# the directory being added is like 'C:/Users/Blackberry/xds/xds_python' or
# '/home/xds/xds_python'
```
to add the xds_python codes in your `Python` path.

Please note that under the folder `xds` there are two subfolders: `xds_matlab` and `xds_python`, in the codes above what has been added into the path is `xds_python`.

Once being added to the path, you need to import the `lab_data` class from `xds` in order to proceed:
```
from xds import lab_data
```

## How to load a data file?

Just use the constructor of `lab_data` to create an `xds` instance. You need to provide **the path to the data file** and **the name of the data file**.

In [15]:
from xds import lab_data
data_path = 'the path to your data files'
file_name = 'Jango_20150730_001.mat'
my_xds = lab_data(data_path, file_name) # Load the data using the lab_data class in xds.py
print('Are there EMGs? %d'%(my_xds.has_EMG))
print('Are there cursor trajectories? %d'%(my_xds.has_cursor))
print('Are there forces? %d'%(my_xds.has_force))

print('\nThe units names are %s'%(my_xds.unit_names))
if my_xds.has_EMG:
    print('\nThe muscle names are %s'%(my_xds.EMG_names))

Trials with nan timings have been removed!
Are there EMGs? 1
Are there cursor trajectories? 1
Are there forces? 1

The units names are ['elec93', 'elec92', 'elec94', 'elec95', 'elec75', 'elec96', 'elec85', 'elec97', 'elec86', 'elec98', 'elec87', 'elec88', 'elec77', 'elec99', 'elec66', 'elec89', 'elec76', 'elec90', 'elec67', 'elec79', 'elec58', 'elec80', 'elec78', 'elec70', 'elec68', 'elec60', 'elec69', 'elec50', 'elec59', 'elec40', 'elec49', 'elec100', 'elec83', 'elec84', 'elec73', 'elec74', 'elec63', 'elec64', 'elec53', 'elec54', 'elec43', 'elec55', 'elec44', 'elec45', 'elec33', 'elec46', 'elec34', 'elec65', 'elec24', 'elec56', 'elec35', 'elec47', 'elec25', 'elec57', 'elec26', 'elec36', 'elec27', 'elec37', 'elec28', 'elec38', 'elec29', 'elec48', 'elec19', 'elec39', 'elec81', 'elec82', 'elec71', 'elec72', 'elec61', 'elec62', 'elec51', 'elec52', 'elec41', 'elec42', 'elec31', 'elec32', 'elec21', 'elec22', 'elec11', 'elec12', 'elec2', 'elec23', 'elec3', 'elec13', 'elec4', 'elec14', 'elec1

`my_xds` is the `lab_data` instance created by the codes and contains all the data, including
* spike timings
* spike counts in consecutive time windows
* unit name or electrode name (depend on spike sorted or not)
* cursor trajectories (if any)
* EMGs (if any)
* muscle names for EMGs (if any)
* forces (if any)

By default, the spike counts have been calculated by binning in 1 ms bins, and synchronized with EMG, force or cursor trajectory data (if any).

## How can I re-bin the data using a different time bin?

Please use the codes below

In [16]:
new_bin_size = 0.05 # the unit is in seconds, here 0.05 s is 50 ms
my_xds.update_bin_data(new_bin_size)  # Bin the spikes with the specified bin_size

The new bin size is 0.0500 s


All the data, including EMGs, forces, and cursor trajectories (if any), have been re-binned using the codes above.

After the binning, you can also smooth the spike counts to get a smooth estimation of the neural firing rates.

In [17]:
smooth_size = 0.1 # the S.D. of the smooth kernel
my_xds.smooth_binned_spikes(new_bin_size, 'gaussian', smooth_size) # Smooth the binned spike counts

## How can I split the data into trials?

The trial based analysis are necessary in most cases. Here we provide these functions to enable the extraction of data in trials:
* get_trials_data_spike_counts( my_type, trial_start, time_ahead, end_event, end_time_offset )
* get_trials_data_EMG( my_type, trial_start, time_ahead, end_event, end_time_offset, EMG_channels = 'all' )
* get_trials_data_force( my_type, trial_start, time_ahead, end_event, end_time_offset )
* get_trials_data_cursor( my_type, trial_start, time_ahead, end_event, end_time_offset )

These functions share most of the input parameters:
* my_type: a character specifying the type of trials:
    * 'R': rewarded trials or successful trials
    * 'F': failed trials
    * 'A': aborted trials
* trial_start: a string specifying the start event when splitting the data into trials, please select among:
    * 'start_time': the start time of trials by the definition of the behavioral paradigm
    * 'gocue_time': the time when the gocue appears
    * 'force_onset_time': the time when the onset of force was detected
    * 'movement_onset_time': the time when the onset of movement was detected (use this when there is no force recordings)
    * 'end_time': the end time of trials by the definition of the behavioral paradigm
* time_ahead: a number, the amount of time you want to take before the trial_start event, can be negative
* end_event: a string specifying the end event when splitting the data into trials, same as `trial_start`
* end_time_offset: a number, the amount of time you want to take after the end_event, can be negative

For the function `get_trials_data_EMG`, there is an additional parameter specifying if picking all or a subset of EMG channels:
* EMG_channels: by default, it's 'all'. If you want to select a subset of EMGs, please provide a list like `['EMG_FCU', 'EMG_EDCr', 'EMG_ECU']`.

The outputs of `get_trials_data_spike_counts`, `get_trials_data_EMG`, `get_trials_data_force` are all `list`, where each element in the list is a `numpy` array containing the data from one trial. After the trial extraction, all the data are still perfectly synchronized on a trial basis.

The outputs of `get_trials_data_cursor` is a list of three lists, corresponding to position, velocity and acceleration. The elements in each list are `numpy` arrays for trials.

So, if you want to extract the portion exactly between **'start_time'** and **'end_time'** for all successful trials, please use these codes:

In [24]:
spike_counts = my_xds.get_trials_data_spike_counts('R', 'start_time', 0.0, 'end_time', 0.0)
EMG = my_xds.get_trials_data_EMG('R', 'start_time', 0.0, 'end_time', 0.0, EMG_channels = 'all')
force = my_xds.get_trials_data_force('R', 'start_time', 0.0, 'end_time', 0.0)
#--------------------- cursor trajectories ---------------------------- #
position = my_xds.get_trials_data_cursor('R', 'start_time', 0.0, 'end_time', 0.0)[0]
velocity = my_xds.get_trials_data_cursor('R', 'start_time', 0.0, 'end_time', 0.0)[1]
acceleration = my_xds.get_trials_data_cursor('R', 'start_time', 0.0, 'end_time', 0.0)[2]

print('The type of spike_counts is %s'%type(spike_counts))
print('The type of EMG is %s'%type(EMG))

The type of spike_counts is <class 'list'>
The type of EMG is <class 'list'>


In the case above the lengths of the trials are different.

If you want to align the trials according to a specific event like 'gocue', and to take **0.5s before the event** and **1s after the event**, please use the codes below:

In [19]:
spike_counts = my_xds.get_trials_data_spike_counts('R', 'gocue_time', 0.5, 'gocue_time', 1.0)
EMG = my_xds.get_trials_data_EMG('R', 'gocue_time', 0.5, 'gocue_time', 1.0, EMG_channels = 'all')
force = my_xds.get_trials_data_force('R', 'gocue_time', 0.5, 'gocue_time', 1.0)

In the case above the lengths of the trials are the same.

Further, if you want align and extract the trials according to **force onset time**, please call the function **`compute_force_onset_time`** first. 

In [20]:
my_xds.compute_force_onset_time()
spike_counts = my_xds.get_trials_data_spike_counts('R', 'force_onset_time', 0.5, 'force_onset_time', 1.0)
EMG = my_xds.get_trials_data_EMG('R', 'force_onset_time', 0.5, 'force_onset_time', 1.0, EMG_channels = 'all')
force = my_xds.get_trials_data_force('R', 'force_onset_time', 0.5, 'force_onset_time', 1.0)

Get the force onset time!


## How can I get the information for the trials?

We provided a function `get_trial_info` to do this job:
* get_trial_info(my_type = 'R')

It will return the information of trials of a specific type ('R', 'F' or 'A') in a list, where you can find information like the direction of the target, the corners and centers of the target for each trial. 

## Can I sort the trials according to specific trial conditions?

To do this job we provided two functions:
* sort_trials_target_dir(data_type, target_dir_list, trial_type, time_params, EMG_channels = 'all')
* sort_trials_target_center(x_or_y, data_type, target_center_list, trial_type, time_params, EMG_channels = 'all')

Most inputs are the same between them, please check the details below:
* data_type: a string specifying the type of data you want, please select among these:
    * 'spike_counts': if you want spike counts
    * 'EMG': if you want EMGs
    * 'force': if you want forces
    * 'cursor': if you want the position, velocity and acceleration of the cursor trajectories
* target_dir_list: a list of target directions you want to include.
* trial_type: 'R', 'F', or 'A', see cells above.
* time_param: a dictionary specifying the events and time to extract and align trials, please check the example codes below.
* EMG_channels: 'all' or a list containing the names of EMG channels you want.

For `sort_trials_target_center` there are two inputs that are different from `sort_trials_target_dir`:
* x_or_y: a character specifying if you want to use the **x coordinates** or **y coordinates** of the target centers to sort the trials, can be 'x' or 'y'.
* target_center_list: a list of target centers (eigher x coordinates or y coordinates) you want to include.

Here let's sort trials according to the directions of the targets, and align trials according to the gocue.

In [36]:
time_params = {
    'start_event': 'gocue_time',
    'time_before_start': 0.5, # the time before the start event is 0.5s
    'end_event': 'gocue_time',
    'time_after_end': 1 # the time after the end event is 1s
}

target_dir_list = list(set(my_xds.trial_target_dir)) # get a list for the directions of the targets across all the trials
print('The directions of the targets in this dataset are %s degrees.'%(target_dir_list))
trial_type = 'R'

spike_counts = my_xds.sort_trials_target_dir('spike_counts', target_dir_list, trial_type, time_params)
print('The length of the list spike_counts is %d'%(len(spike_counts)))
EMG = my_xds.sort_trials_target_dir('spike_counts', target_dir_list, trial_type, time_params, EMG_channels = 'all')
print('The length of the list EMG is %d'%(len(EMG)))

The directions of the targets in this dataset are [0.0, -90.0, 135.0, 45.0, -45.0, 180.0, -135.0, 90.0] degrees.
The length of the list spike_counts is 8
The length of the list EMG is 8


Since the targets are in 8 different directions in this dataset, the lengths of the extracted lists `spike_counts` and `EMG` are 8, where each element is a list containing the trials for this target direction.

## What if I want continuous recordings?

Please use the codes below:

In [25]:
cont_time_frame = my_xds.time_frame
cont_spike_counts = my_xds.spike_counts
cont_EMG = my_xds.EMG

Then you can use `cont_time_frame` to help select a portion of data.